In [ ]:
import networkx as nx
import pandas as pd
import json
from typing import DefaultDict
import matplotlib.pyplot as plt
from pyvis.network import Network
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# Companion Planting

## Objective
Create and visualize a network of companion plants to make planting recommendations. 

In [ ]:
path = '../data/gardenplanner.json'
        
def get_plants(path):
    english_data =  DefaultDict(dict)
    with open(path) as json_file:
        data = json.load(json_file)
        for plant in data['plants']:
            if plant['countryCode'] == 'gb':
                english_data[plant['plantCode']] = plant
                     
        return english_data
    
def get_relationships(data, edibles_only = False):
    code_dict = {}
    relationship_dict = DefaultDict(set)
        
    edibles = ['fruit', 'herb', 'vegetable']

    for plant in data:
        code_dict[plant] = data[plant]['plantName']

        if edibles_only:
            if any(item in data[plant]['tags'] for item in edibles):
                relationship_dict[plant] = set(data[plant]['companionPlantCodes'])
        else:
            relationship_dict[plant] = set(data[plant]['companionPlantCodes'])
                    
    return code_dict, relationship_dict

def print_plant_list(relationship_dict, code_dict):
    for plant in sorted(relationship_dict, key=lambda k: len(relationship_dict[k]), reverse=True):
        print(f'plant: {code_dict[plant]} ({plant}), num_of_companions: {len(relationship_dict[plant])}')
        
data = get_plants(path)
code_dict, relationship_dict = get_relationships(data)
print_plant_list(relationship_dict, code_dict)

Garlic, carrots, nasturtiums, lettuce (loose leaf and headed), tomatoes and kale seem to be the most 'sociable' plants in the database. There are several plants, like ginger, walnuts, persimmon and lemon grass with no companions or only one companion. 

In the future, we can try to get data from other sources, because some of these plants do have known companions. For example, a quick search shows that cherry, pawpaw, persimmon, plum, and quince will grow under walnuts.

The full database also contains flowers, so a quick way to narrow down the data to edibles is to only look at fruits, vegetables and herbs (though this may miss some useful, edible plants like nasturtiums).

In [ ]:
g_all = nx.MultiGraph(relationship_dict)
net = Network(notebook = True, width = 1000)
net.from_nx(g_all)
net.show_buttons()
net.show("companion.html")

In [ ]:
# show all plant connections
def create_pyvis_network(graph, show_buttons = True):
    net = Network(notebook = True, width = 1000)
    net.from_nx(graph)
    if show_buttons:
        net.show_buttons()
    return net
    
n = create_pyvis_network(g_all, "all_plant_connections.html")
n.show("../output/all_plant_connections.html")

In [ ]:
#graph of only veggies, fruits and herbs
code_dict_edibles, relationship_dict_edibles = get_relationships(data, edibles_only=True)
g_edible = nx.MultiGraph(relationship_dict_edibles)
n = create_pyvis_network(g_edible)
n.show("../output/edible_plant_connections.html")

In [ ]:
#calculate cosine similarity
companion_matrix = nx.to_numpy_matrix(g_all)
sim_matrix = cosine_similarity(companion_matrix, companion_matrix)
sim_df = pd.DataFrame(sim_matrix, columns = g_all.nodes, index = g_all.nodes)

most_similar_ix = cosine_similarity(companion_matrix[i:i+1], companion_matrix).flatten().argsort()

In [ ]:
# df = pd.DataFrame(columns=["original df col", "most similar doc", "similarity%"])
# for i in range(len(g_all.nodes)):
#     cosine_similarities = cosine_similarity(companion_matrix[i:i+1], companion_matrix).flatten()
#     # make pairs of (index, similarity)
#     cosine_similarities = list(enumerate(cosine_similarities))
#     # delete the cosine similarity with itself
#     cosine_similarities.pop(i)
#     # get the tuple with max similarity
#     most_similar, similarity = max(cosine_similarities, key=lambda t:t[1])
#     df.loc[len(df)] = [g_all.nodes[i], g_all.nodes[most_similar], similarity]

In [ ]:
#TODO: give 3 degrees of connections
#TODO: Add nigrogen needs/production and insectary plant info
#TODO: get/clean sun, soil, water needs